<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_0_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    689      0 --:--:-- --:--:-- --:--:--   689
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.127520,4.244538,0.267889,04:19


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.044175,3.708712,0.333355,04:18


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.742898,3.668341,0.338138,04:19


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.705012,3.634060,0.343367,04:22


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.643473,4.336130,0.315976,01:14


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.867755,3.984721,0.363314,01:28


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.527504,3.814691,0.386982,03:51


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.345738,3.703298,0.402367,03:54
1,3.163604,3.618709,0.409467,03:40


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.095185,3.526966,0.418935,03:52
1,2.971253,3.483069,0.420118,03:44


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.932857,3.411802,0.431953,03:58
1,2.842841,3.357440,0.436686,03:32


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.766759,3.388111,0.439053,03:38
1,2.634425,3.344736,0.449704,03:56


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.643276,3.366406,0.447337,03:35
1,2.601303,3.276114,0.463905,03:54


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.648485,3.304714,0.461538,03:57
1,2.532645,3.279458,0.461538,03:53
2,2.498573,3.243309,0.467456,03:49
3,2.427499,3.227440,0.468639,03:53


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.463228,3.247144,0.463905,03:39
1,2.433108,3.215660,0.467456,03:38
2,2.421554,3.284741,0.466272,03:45
3,2.309927,3.258810,0.471006,03:31


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.378348,3.279380,0.473373,03:47
1,2.474571,3.358509,0.475740,03:51
2,2.371575,3.204454,0.479290,03:37
3,2.257780,3.200793,0.475740,04:02


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.294172,3.245461,0.469822,03:31
1,2.343210,3.347279,0.471006,03:50
2,2.280906,3.201379,0.475740,04:02
3,2.207441,3.248631,0.476923,03:42


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.198214,3.356568,0.479290,03:48
1,2.307665,3.371686,0.473373,03:32
2,2.222464,3.291179,0.479290,03:50
3,2.267174,3.387377,0.478106,03:42


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.217426,3.369663,0.472189,03:48
1,2.325510,3.348014,0.475740,03:48
2,2.174021,3.323615,0.484024,03:30
3,2.153545,3.379742,0.480473,03:29


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.105761,3.376527,0.482840,03:38
1,2.225516,3.318534,0.481657,03:51
2,2.184772,3.381139,0.478106,03:53
3,2.144347,3.421223,0.481657,03:38


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[49, 864, 210, 334, 258, 1026, 1026, 1026, 1026, 1026, 1026, 1026, 123, 258, 517, 49, 555, 1035, 27, 644, 29, 76, 211, 864, 31, 414, 32, 34, 197, 864, 1035, 289, 864, 864, 864, 210, 218, 44, 1026, 49, 49, 49, 49, 495, 49, 218, 853, 853, 853, 853, 853, 853, 853, 853, 53, 753, 289, 61, 1035, 896, 491, 76, 76, 182, 741, 111, 111, 190, 517, 517, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 713, 289, 289, 517, 517, 80, 80, 80, 80, 80, 80, 80, 80, 80, 1035, 89, 892, 137, 27, 753, 1026, 94, 94, 517, 94, 94, 94, 94, 94, 218, 360, 746, 1035, 246, 444, 806, 27, 644, 874, 100, 100, 100, 896, 960, 257, 289, 524, 1035, 864, 636, 624, 864, 864, 864, 1035, 1035, 773, 117, 831, 517, 517, 1026, 125, 517, 125, 777, 777, 777, 777, 777, 777, 777, 778, 777, 343, 127, 131, 1035, 447, 447, 164, 96, 415, 517, 847, 182, 758, 624, 854, 178, 165, 245, 180, 517, 291, 289, 289, 864, 864, 864, 864, 864, 686, 182, 864, 864, 864, 

In [44]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

201
0.23183391003460208
